# Lab session 4: Data Warehousing and On-line Analytical Processing

## Introduction 

The aim of this lab is for students to get experience with **Data Warehousing** and **On-line Analytical Processing (OLAP)** covered in week 5, and more specifically with the concepts of **data cubes**, **data cube measures**, **typical OLAP operations**, and **data cube computation**.

- This lab is the first part of a **two-week assignment** that covers weeks 5 and 6, which is due on **Friday 12th November 10am**.
- The assignment will account for 10% of your overall grade. Questions in this lab sheet will contribute to 5% of your overall grade; questions in the lab sheet for week 6 will cover for another 5% of your overall grade.
- <font color = 'maroon'>The last section of this notebook includes the questions that are assessed towards your final grade.</font> 

## Important notes about the assignment: 

- **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!). 
- The total assessed coursework is worth 40% of your final grade.
- There will be 9 lab sessions and 4 assignments.
- One assignment will cover 2 consecutive lab sessions and will be worth 10 marks (percentages of your final grade).
- The submission cut-off date will be 7 days after the deadline and penalties will be applied for late submissions in accordance with the School policy on late submissions.
- You are asked to submit a **report** that should answer the questions specified in the last section of this notebook. The report should be in a **single PDF document** (so **NOT** *doc, docx, notebook* etc). This single PDF document will include your answers to both the week 5 and week 6 labs.
- No other means of submission other than submitting your assignment through the appropriate QM+ link are acceptable at any time. Submissions sent via email will **not** be considered.
- Please name your report as follows: Assignment2-StudentName-StudentNumber.pdf
- Cases of **Extenuating Circumstances (ECs)** have to go through the proper procedure of the School in due time. Only cases approved by the School in due time can be considered.

## 1. Introduction to Cubes

This chapter describes step-by-step how to use Cubes (http://cubes.databrewery.org/), a lightweight Python framework and set of tools for development of reporting and analytical applications, Online Analytical Processing (OLAP), multidimensional analysis and browsing of aggregated data. We will be working with v1.1 of Cubes. Cubes features:
- a logical view of analysed data - how analysts look at data, how they think of data, not not how the data are physically implemented in the data stores
- OLAP and aggregated browsing (default backend is for relational databse - ROLAP)
- hierarchical dimensions (attributes that have hierarchical dependencies, such as category-subcategory or country-region)
- multiple hierarchies in a dimension
- localizable metadata and data (see Localization)
- authentication and authorization of cubes and their data
- pluggable data warehouse – plug-in other cube-like (multidimensional) data sources

Cubes is meant to be used by application builders that want to provide analytical functionality. Cubes also relies on methods from SQLAlchemy (https://www.sqlalchemy.org/), an open-source SQL toolkit and object-relational mapper for Python.

## 2. Data Preparation

The example data used here is the International Bank for Reconstruction and Development (IBRD) Balance Sheet. For this example we will be using the CSV file "IBRD_Balance_Sheet__FY2010.csv" which is provided in the supplementary material for the lab. The CSV file includes records which are characterised by a Category (and subcategories), Line Item, Fiscal Year, and Amount (in US$ millions). We first start with imports:

In [12]:
# If you are using Google Colab, you would need to run the below line to install Cubes. 
# You can comment the below line if you are running a local python installation with Cubes installed.
# The below line also installs a particular version of the sqlalchemy package needed to use Cubes.
# !pip install cubes
# !pip install -Iv sqlalchemy==1.3.9

from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

We can now load the data, create a table and populate it with contents of the CSV file. Note the categories and subcategories created; check the CSV file to link the below commands with the contents of the dataset.

In [13]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

## 3. Creating a data cube

Everything in Cubes happens in an *analytical workspace*. It contains cubes, maintains connections to the data stores (with cube data), provides connection to external cubes and more.

The workspace properties are specified in a configuration file slicer.ini (default name). The first thing we have to do is to specify a data store – a database which will host the cube’s data:

In [14]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

AttributeError: module 'sqlalchemy.util' has no attribute 'dependencies'

The structure of data cubes (in terms of dimensions, measures, and aggregates) is specified in JSON files. We now import file 'tutorial_model.json' (found in the lab supplementary material) which includes an example model of the data cube, dimension tables, and aggregate functions for the CSV file we loaded previously.

In [ ]:
workspace.import_model("tutorial_model.json")

**Please make sure to inspect the structure of the above JSON file - this will be relevant for one of the assignment questions.**

We can now create a data cube based on the above data cube model and data table:

In [ ]:
cube = workspace.cube("ibrd_balance")

## 4. Aggregations and OLAP operations

*Browser* is an object that does the actual aggregations and other data queries for a cube. To obtain one:

In [ ]:
browser = workspace.browser(cube)

We can now compute aggregates of the data cube as specified by the data cube model. For computing the total count of records:

In [ ]:
result = browser.aggregate()
result.summary["record_count"]

For computing a sum of the amount:

In [ ]:
result.summary["amount_sum"]

Now we can try to compute aggregates by year:

In [ ]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

Or compute aggregates by item category:

In [ ]:
result = browser.aggregate(drilldown=["item"])
for record in result:
    print(record)

We can also perform *slicing* and *dicing* operations on the data cube. The below example performs a slicing operation on the data cube by selecting only entries with the year being 2009, and displays aggregates according to the item category. Here, a *cell* defines a point of interest – portion of the cube to be aggergated or browsed.

In [ ]:
import cubes as cubes
cuts = [cubes.PointCut("year", ["2009"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

It's worth noting that in Cubes, slicing operations can be created by either specifying a "point cut" which selects a single value of an attribute in a given dimension (called using the cubes.PointCut() function as above), or by specifying a "range cut", which selects a range of values for a given dimension. The range cut can be called using the cubes.RangeCut() function, which takes as input the attribute name, the minimum value of the specified range, and the maximum value of the range.

Similarly, we can perform a *dicing* operation on the data cube by performing a selection on two or more dimensions. The below example performs a dicing operation on the data cube, selecting entries with the year being 2009 and the item category being "a", and displays the aggregate results:

In [ ]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item"])
result.summary

We can also *drill down* lower in the Category hierarchy. Here, we perform a dicing operation to select records with year being 2009 and item category being "a" (corresponding to assets), and show aggregates for each subcategory level.

In [ ]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item:subcategory"])
for record in result:
    print(record)

## <font color = 'maroon'>Assignment 2 [part 1 of 2]</font>

Questions 1-2 are pen-and-paper exercises; questions 3-4 are coding exercises. For all your answers to the assignment, please include include your workings (e.g. equations, code) when this is relevant to the question.

1. A data warehouse for a music streaming company consists of the dimensions song, user, time (time and date of when the user listened to a song), and the two measures count (how many times a user listened to the song) and fee (fee paid by the streaming company to the artist every time a user listens to that song). 
  1. Draw a schema diagram for the above data warehouse using a star schema. [1 mark out of 5]
  2. Starting with the base cuboid [time, user, song], what specific OLAP operations should be performed in order to list the total fee collected for a given song for a given month of a given year (e.g. October 2021)? [0.5 marks out of 5]
  3. Assume that the time dimension has 4 levels: day, month, quarter, year; and that the song and user dimensions both have 1 level (not including the virtual level 'all'). How many cuboids will this cube contain (including the base and apex cuboids)? [0.5 marks out of 5]
  

2. Suppose that a car rental company has a data warehouse that holds record ID lists of vehicles in terms of brands (Audi, Ford, Mini) and store branches (Tower Hamlets, Newham, Hackney). Each record consists of a combination of vehicle brand and branch, and records for all combinations exist. We would like to index the OLAP data using bitmap indices. Write down the *base table* for record IDs, and the corresponding *bitmap index table* for vehicle brand. [0.5 marks out of 5]
  

3. Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your workings in the PDF report. You can read the [Cubes package documentation](https://readthedocs.org/projects/cubes/downloads/pdf/latest/) for assistance in this task. [1 mark out of 5]


4. Using the CSV file "country-income.csv" (found in the week 5 supplementary lab documents), perform the following:
  1. Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files created. [0.5 marks out of 5]
  2. Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 40 and 50. [1 mark out of 5]
  